In [1]:
import os
import sys
import re
from nltk.corpus import stopwords
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
from pyspark.sql import SparkSession, SQLContext
import os
import pyspark.sql.functions as f
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

22/04/12 19:42:46 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/12 19:42:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/12 19:42:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/12 19:42:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/12 19:42:48 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/04/12 19:42:48 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
sc = SparkSession.builder.appName('App Name').master("local[*]").getOrCreate()
sqlContext = SQLContext(sparkContext=sc.sparkContext, sparkSession=sc)

In [4]:
data = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load(os.path.realpath("data/fanfics_raw.csv"))

In [5]:
import nltk
nltk.pos_tag(["Arthur", "loves", "fish"])

[('Arthur', 'NNP'), ('loves', 'VBZ'), ('fish', 'JJ')]

In [6]:
import pyspark.sql.functions as f
data = data.withColumn('line', f.explode(f.split(f.col('text'), '\n')))
data.show()

+--------------------+--------------------+--------------------+--------------------+
|                  id|                text|                type|                line|
+--------------------+--------------------+--------------------+--------------------+
|             1536152|           Chapter 1|                null|           Chapter 1|
|Going back was th...| after a case lik...| I’d be moved to ...| after a case lik...|
|It was during my ...| Sophia was no lo...| especially since...| Sophia was no lo...|
|Following Sophia ...| I ducked in an e...| waiting under a ...| I ducked in an e...|
|Aegis breaking in...| but when they ca...| barely managing ...| but when they ca...|
|I hated coming to...|                sure| but everyone her...|                sure|
|             Somehow| this time was ev...| my instincts wer...| this time was ev...|
|The restaurant Li...| just off the Boa...| and without thos...| just off the Boa...|
|I managed to snea...| I tested for spe...| there were

In [7]:
def removeProperNouns(sentence):
    tagged_sentence = nltk.tag.pos_tag(nltk.word_tokenize(sentence))
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'NNP' and tag != 'NNPS']
    return ' '.join(edited_sentence)

In [8]:
classics = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load(os.path.realpath("data/classics_raw.csv"))
StopWords = nltk.corpus.stopwords.words("english")
classics = data.rdd.map(lambda x : x['text']).filter(lambda x: x is not None)
tokens = classics                                                 \
    .map(lambda document: removeProperNouns(document))             \
    .map( lambda document: document.strip().lower())               \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])     \
    .zipWithIndex()

In [ ]:
from pyspark.sql.functions import udf, col, StringType, column
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, IDF
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=1.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 

In [ ]:
from pyspark.sql.functions import udf, col, StringType, column
from pyspark.ml.feature import RegexTokenizer, CountVectorizer
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=1.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 

In [ ]:
num_topics = 10
max_iterations = 1000
lda = LDA(k=num_topics, maxIter=max_iterations)
lda_model = lda.fit(result_tfidf[['index','features']])

In [ ]:
data.printSchema()
data.show()

In [ ]:
from pyspark.sql.types import *
vocab = cvmodel.vocabulary
vocab_broadcast = sc.sparkContext.broadcast(vocab)

#creating LDA model
ldatopics = lda_model.describeTopics()

def map_termID_to_Word(termIndices):
    words = []
    for termID in termIndices:
        words.append(vocab_broadcast.value[termID])

    return words

udf_map_termID_to_Word = udf(map_termID_to_Word , ArrayType(StringType()))

ldatopics_mapped = ldatopics.withColumn("topic_desc", udf_map_termID_to_Word(ldatopics.termIndices))

In [ ]:
ldatopics_mapped.show()

In [ ]:
from pyspark.sql.functions import concat_ws, split
import pyspark.sql.functions as f
lda_topics_mapped = ldatopics_mapped.withColumn("topic_desc", concat_ws(" ", ldatopics_mapped.topic_desc).alias("topic_desc_str"))

lda_topics_mapped.show()

In [ ]:
counts = lda_topics_mapped.withColumn('word', f.explode(f.split(f.col('topic_desc'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=True)
counts.show()

In [ ]:
import pyspark.sql.functions as F
from collections import ChainMap
from wordcloud import WordCloud
import matplotlib.pyplot as plt
wordcloud = WordCloud(background_color="white")

words = dict(ChainMap(*counts.select(F.create_map('word', 'count')).rdd.map(lambda x: x[0]).collect()))
# {'scorbutically': 1.76, 'punta': 1.76, 'detail': 1.789, 'lafayette': 1.8, 'maya': 1.854, 'prostate': 1.854, 'quot': 1.856, 'mark': 1.949, 'elite': 1.988, 'trade': 2.012, 'write': 2.083}

plt.imshow(wordcloud.generate_from_frequencies(words))

plt.show()


In [ ]:
nltk.pos_tag(["lisa"])

In [ ]:
removeProperNouns("Lisa is me")